In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [84]:
with open('./all_docs_results.json') as json_file:
    predictions = json.load(json_file)
    
with open('./project-12-at-2021-07-15-14-39-0020e5b8.json') as json_file:
    ground_truth = json.load(json_file)

## Predictions
### Exploración

In [85]:
predictions['11821']['CSF_match_results'].keys()

dict_keys(['state_csf', 'society_type_csf', 'company_name_csf'])

In [86]:
predictions['11821']['ROIS_qa_results'].keys()

dict_keys(['notario', 'date'])

In [87]:
society_type_csf = predictions['11821']['CSF_match_results']['society_type_csf'][3]
company_name_csf = predictions['11821']['CSF_match_results']['company_name_csf'][3]
notario = predictions['11821']['ROIS_qa_results']['notario'][3]
date = predictions['11821']['ROIS_qa_results']['date'][3]

## Armo un df de predicciones

In [88]:
campos = []
for i in range(len(predictions)):
    for k in predictions['11821']['CSF_match_results'].keys():
        campos.append(k)
    for k in predictions['11821']['ROIS_qa_results'].keys():
        campos.append(k)
        
campos_unique = sorted(set(campos))
campos_unique

['company_name_csf', 'date', 'notario', 'society_type_csf', 'state_csf']

In [89]:
df_preds = pd.DataFrame(columns = ['documento'])
for c in campos_unique:
    df_preds[c] = []
    
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf


In [90]:
i = 0
for k in predictions.keys():
    
    df_preds.loc[i, 'documento'] = k
    
    # a veces puede ser que 'CSF_match_results' o 'ROIS_qa_results' estén vacíos
    try:
        for j in predictions[k]['CSF_match_results'].keys():
            try:
                df_preds.loc[i, j] = predictions[k]['CSF_match_results'][j][3]
            except:
                continue
    except:
        continue
        
    try:    
        for j in predictions[k]['ROIS_qa_results'].keys():
            try:
                df_preds.loc[i, j] = predictions[k]['ROIS_qa_results'][j][3]
            except:
                continue
    except:
        continue
        
    i = i+1
            
df_preds.head()

,documento,company_name_csf,date,notario,society_type_csf,state_csf
0,11821,GRUPO CYCMP,veintiumo de julio del dos mil ocho,MAURICIO MIER PADRÓN,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,San Luis Potosi
1,12078,BITUBI CONSULTING,trece de febrero,JORGE JULIÁN PEREYRA ESCUDERO,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,Quintana Roo
2,12192,DESARROLLO DE SOFTWARE MONOBITS,22 de noviembre de 2012,YO,Sociedad de Responsabilidad Limitada de Capita...,BAJA CALIFORNIA
3,12264,CHARVEL Y PEZUELA GROUP,dos de octubre,NaN,SOCIEDAD ANONIMA DE CAPITAL VARIABLE,CIUDAD DE MÉXICO
4,12286,EXELENCIA EJECUTIVA PREMIUM,NaN,RAFAEL OVARRUBIAS FLORES,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,JALISCO


De todos los campos que puedo encontrar en las actas, no todos los puedo validar, sólo éstos de arriba

## Ground truth
### Exploración

In [91]:
# este tiene la boleta de inscripción
ground_truth[0]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.866666666666667,
   'y': 11.902834008097166,
   'width': 83.46666666666667,
   'height': 36.51821862348178,
   'rotation': 0,
   'rectanglelabels': ['boleta_inscripcion']},
  'id': 'nVvt8xx7hA',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'choices': ['yes']},
  'id': 'r_7Rb71M-M',
  'from_name': 'has_boleta_inscripcion',
  'to_name': 'image',
  'type': 'choices'}]

In [92]:
# este tiene el cuadro accionario y el apoderado
ground_truth[1]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.432706222865413,
   'y': 39.19156414762742,
   'width': 57.45296671490593,
   'height': 7.0298769771529,
   'rotation': 0,
   'rectanglelabels': ['sh_table_lines']},
  'id': 'ANsPu48gOp',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'text': ['Jorge Cañedo Osuna']},
  'id': 'asa879Jesq',
  'from_name': 'apoderado',
  'to_name': 'image',
  'type': 'textarea'}]

In [93]:
# este tiene varios campos
# lo que me interesa es todo lo que está en "result"
result_lista = ground_truth[2]['annotations'][0]['result']

result_lista

[{'value': {'text': ['Miguel Enrique Martinez Serrato']},
  'id': 'KdMY7vBSEi',
  'from_name': 'notario',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Compass4PL']},
  'id': 'hQFu6ebrse',
  'from_name': 'razon_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['S de RL de cv']},
  'id': 'P7GI5OarC-',
  'from_name': 'denominacion_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['27/06/2015']},
  'id': '_LjHuXTpo8',
  'from_name': 'fecha_inscripcion',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['12236']},
  'id': 'cSq9FwAKry',
  'from_name': 'instrumento_publico',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Navojoa Sonora Mexico']},
  'id': 'XAR0_Uu-DH',
  'from_name': 'direccion',
  'to_name': 'image',
  'type': 'textarea'}]

In [94]:
id = ground_truth[1]['id']
id

7887

In [95]:
documento = ground_truth[1]['file_upload'].split('-')[0]
documento

'16519'

In [96]:
# la página a veces tiene un "_" seguido de otras cosas
pagina = ground_truth[1]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
pagina

'014'

In [97]:
# acá no hay campos que me interesen
ground_truth[0]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.866666666666667,
   'y': 11.902834008097166,
   'width': 83.46666666666667,
   'height': 36.51821862348178,
   'rotation': 0,
   'rectanglelabels': ['boleta_inscripcion']},
  'id': 'nVvt8xx7hA',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'choices': ['yes']},
  'id': 'r_7Rb71M-M',
  'from_name': 'has_boleta_inscripcion',
  'to_name': 'image',
  'type': 'choices'}]

In [98]:
# acá me interesa el campo "apoderado"
ground_truth[1]['annotations'][0]['result']

[{'original_width': 1024,
  'original_height': 1686,
  'image_rotation': 0,
  'value': {'x': 11.432706222865413,
   'y': 39.19156414762742,
   'width': 57.45296671490593,
   'height': 7.0298769771529,
   'rotation': 0,
   'rectanglelabels': ['sh_table_lines']},
  'id': 'ANsPu48gOp',
  'from_name': 'label',
  'to_name': 'image',
  'type': 'rectanglelabels'},
 {'value': {'text': ['Jorge Cañedo Osuna']},
  'id': 'asa879Jesq',
  'from_name': 'apoderado',
  'to_name': 'image',
  'type': 'textarea'}]

Vemos que el primer key es "value".

Acá hay varios campos que podría levantar, aunque no son los 4 que tengo en el otro archivo. También empiezan con "value":

In [99]:
ground_truth[2]['annotations'][0]['result']

[{'value': {'text': ['Miguel Enrique Martinez Serrato']},
  'id': 'KdMY7vBSEi',
  'from_name': 'notario',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Compass4PL']},
  'id': 'hQFu6ebrse',
  'from_name': 'razon_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['S de RL de cv']},
  'id': 'P7GI5OarC-',
  'from_name': 'denominacion_social',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['27/06/2015']},
  'id': '_LjHuXTpo8',
  'from_name': 'fecha_inscripcion',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['12236']},
  'id': 'cSq9FwAKry',
  'from_name': 'instrumento_publico',
  'to_name': 'image',
  'type': 'textarea'},
 {'value': {'text': ['Navojoa Sonora Mexico']},
  'id': 'XAR0_Uu-DH',
  'from_name': 'direccion',
  'to_name': 'image',
  'type': 'textarea'}]

In [100]:
# para levantar los campos anteriores:

# me molesta también el caso {'value': {'choices': ['yes']}
# me interesan sólo los que dicen {'value': {'text': ['Miguel Enrique Martinez Serrato']}
# así que quiero descartar los que dicen distinto de "text"
# si no logra acceder que lo saltee

# lista = ground_truth[2]['annotations'][0]['result']

# for l in lista:
#     # acá salteo todos los que no empiecen con "value"
# #     if list(l.keys())[0] != 'value':
# #         continue
#     try:
#         print({l['from_name']: l['value']['text'][0]})
#     except:
#         continue

In [101]:
# result_lista = ground_truth[1]['annotations'][0]['result']
# for r in result_lista:
#     print(r)
    
# print()

# for r in result_lista:
#     # por ahora busco sólo campos de texto, así que los recuadros los salteo
#     if list(r.keys())[0] == 'original_width':
#         continue
#     print(r)

In [102]:
# result_lista = ground_truth[2]['annotations'][0]['result'][0]
# result_lista

In [103]:
# i = 0
# result_lista = ground_truth[i]['annotations'][0]['result']
# campos = []
# for r in result_lista:
#     campos.append({r['from_name']: r['value']['text'][0]})

## Armo un df de ground truth

In [104]:
# quiero saber cuáles son todos los campos posibles para hacer columnas en el df
campos = []
for i in range(len(ground_truth)):
    lista = ground_truth[i]['annotations'][0]['result']
    
    for l in lista:
        # acá salteo todos los que no tengan el formato que me interesa
        try:
            campos.append(l['from_name'])
        except:
            continue
            
campos_unique = sorted(set(campos))

In [105]:
for c in campos_unique:
    print(c)

apoderado
denominacion_social
direccion
fecha_inscripcion
has_boleta_inscripcion
instrumento_publico
label
notario
razon_social


In [106]:
# hago una nueva versión del df con una columna por campo
df = pd.DataFrame(columns = ['id', 'documento', 'pagina'])
for c in campos_unique:
    df[c] = []
    
df.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social


In [107]:
# ahora trato de llenar todos esos campos, cuando es posible

for i in range(len(ground_truth)):
#     print(i)
    identity = ground_truth[i]['id']
    documento = ground_truth[i]['file_upload'].split('-')[0]
    pagina = ground_truth[i]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
    
    lista = ground_truth[i]['annotations'][0]['result']

    campos = []
    for l in lista:
        df.loc[i, 'id'] = identity
        df.loc[i, 'documento'] = documento
        df.loc[i, 'pagina'] = pagina
        
        try:
            df.loc[i, l['from_name']] = l['value']['text'][0]
        except:
            continue
    
df.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
0,7905,16519,022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7887,16519,014,Jorge Cañedo Osuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7885,16519,001,NaN,S de RL de cv,Navojoa Sonora Mexico,27/06/2015,NaN,12236,NaN,Miguel Enrique Martinez Serrato,Compass4PL
3,7861,16490,018,Jorge Antonio Espinoza Barragan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7878,16490,002,NaN,NaN,Cuernavaca Morelos,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# ordeno por documento y página
df = df.sort_values(['documento', 'pagina'])
df.head()

,id,documento,pagina,apoderado,denominacion_social,direccion,fecha_inscripcion,has_boleta_inscripcion,instrumento_publico,label,notario,razon_social
1276,6631,10227,009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,6635,10227,019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,6908,11821,000,NaN,sa de cv,san luis potosi,21/07/2008,NaN,39413,NaN,mauricio mier padron,grupo cycme
95,6897,11821,011,raymundo garza elizondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,6910,11821,015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Comparación entre los 2 df